# Are You Becoming Who You Want to Be?

**Ariata measures the gap between your aspirational values and actual behavior.**

In [19]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Style settings
colors = {
    'family': '#FF6B6B',
    'deep_work': '#4ECDC4',
    'fitness': '#45B7D1',
    'social': '#96CEB4',
    'entertainment': '#FECA57'
}

## 1. Your Stated Values

What you say matters most in life (from onboarding):

In [20]:
# User's stated value hierarchy (1.0 = highest priority)
stated_values = {
    'family': 1.0,        # "Family comes first"
    'deep_work': 0.85,    # "Meaningful career impact"
    'fitness': 0.70,      # "Health is wealth"
    'social': 0.50,       # "Quality friendships"
    'entertainment': 0.20 # "Occasional relaxation"
}

# Visualize stated priorities
fig = go.Figure(data=[
    go.Bar(
        x=list(stated_values.values()),
        y=list(stated_values.keys()),
        orientation='h',
        marker_color=[colors[k] for k in stated_values.keys()],
        text=[f'{v:.0%}' for v in stated_values.values()],
        textposition='auto',
    )
])
fig.update_layout(
    title="What You Say Matters Most",
    xaxis_title="Priority Level",
    height=300,
    showlegend=False
)
fig.show()

## 2. This Week's Actual Behavior

Extracted from your Ariata data streams (location, calendar, screen time, health):

In [21]:
# Actual behavioral events from the past week
weekly_events = [
    # Monday
    {'day': 'Mon', 'time': '6:00 PM', 'duration': 3.0, 'action': 'Watched Netflix', 
     'category': 'entertainment', 'context': 'Skipped family dinner'},
    {'day': 'Mon', 'time': '9:00 PM', 'duration': 0.5, 'action': 'Scrolled social media in bed',
     'category': 'entertainment', 'context': 'Partner wanted to talk'},
    
    # Tuesday  
    {'day': 'Tue', 'time': '6:00 AM', 'duration': 1.0, 'action': 'Gym workout',
     'category': 'fitness', 'context': 'Before work'},
    {'day': 'Tue', 'time': '9:00 AM', 'duration': 3.0, 'action': 'Deep focus coding',
     'category': 'deep_work', 'context': 'Phone in drawer'},
    
    # Wednesday
    {'day': 'Wed', 'time': '5:30 PM', 'duration': 2.0, 'action': 'Kid\'s recital',
     'category': 'family', 'context': 'Declined work drinks'},
    {'day': 'Wed', 'time': '8:00 PM', 'duration': 1.0, 'action': 'Bedtime stories',
     'category': 'family', 'context': 'No devices'},
    
    # Thursday
    {'day': 'Thu', 'time': '10:00 AM', 'duration': 2.0, 'action': 'Twitter/Reddit during "deep work"',
     'category': 'entertainment', 'context': 'Deadline tomorrow'},
    {'day': 'Thu', 'time': '7:00 PM', 'duration': 0.5, 'action': 'Ordered takeout, ate at desk',
     'category': 'deep_work', 'context': 'Family ate without me'},
    
    # Friday
    {'day': 'Fri', 'time': '6:00 PM', 'duration': 3.0, 'action': 'Date night',
     'category': 'family', 'context': 'Left work on time'},
    {'day': 'Fri', 'time': '10:00 PM', 'duration': 1.0, 'action': 'Drinks with friends',
     'category': 'social', 'context': 'After date'},
]

df_events = pd.DataFrame(weekly_events)

# Show this week's time allocation
time_by_category = df_events.groupby('category')['duration'].sum().to_dict()

# Ensure all categories are represented
for cat in stated_values.keys():
    if cat not in time_by_category:
        time_by_category[cat] = 0

print("📊 This Week's Actual Time Allocation:")
print("=" * 40)
total_hours = sum(time_by_category.values())
for category, hours in sorted(time_by_category.items(), key=lambda x: x[1], reverse=True):
    percentage = (hours / total_hours) * 100
    print(f"{category:15} {hours:4.1f}h  ({percentage:5.1f}%)")
print("=" * 40)
print(f"{'Total':15} {total_hours:4.1f}h")

📊 This Week's Actual Time Allocation:
family           6.0h  ( 35.3%)
entertainment    5.5h  ( 32.4%)
deep_work        3.5h  ( 20.6%)
fitness          1.0h  (  5.9%)
social           1.0h  (  5.9%)
Total           17.0h


## 3. Event Embedding & Alignment

Each action becomes a point in value space. Distance from ideal = misalignment.

In [22]:
# Convert events to vectors based on value alignment
def event_to_vector(event, values):
    """Convert an event into a vector based on value alignment"""
    # Base vector from category alignment
    vector = np.zeros(len(values))
    categories = list(values.keys())
    
    # Primary category gets positive weight
    if event['category'] in categories:
        idx = categories.index(event['category'])
        vector[idx] = event['duration']
    
    # Context modifies the vector (trade-offs)
    if 'Skipped family' in event.get('context', ''):
        family_idx = categories.index('family')
        vector[family_idx] = -event['duration'] * 0.5
    
    if 'Declined work' in event.get('context', ''):
        family_idx = categories.index('family')
        vector[family_idx] += event['duration'] * 0.3
        
    if 'during "deep work"' in event.get('context', ''):
        work_idx = categories.index('deep_work')
        vector[work_idx] = -event['duration']
    
    return vector

# Convert all events to vectors
event_vectors = [event_to_vector(e, stated_values) for e in weekly_events]

# Calculate revealed values from actual behavior
revealed_behavior = np.sum(event_vectors, axis=0)
revealed_behavior = np.maximum(revealed_behavior, 0)  # No negative values
revealed_total = np.sum(revealed_behavior)

if revealed_total > 0:
    revealed_values = dict(zip(stated_values.keys(), revealed_behavior / revealed_total))
else:
    revealed_values = {k: 0 for k in stated_values.keys()}

# Normalize to match stated values scale (max = 1.0)
max_revealed = max(revealed_values.values()) if revealed_values else 1
if max_revealed > 0:
    scaling_factor = 1.0 / max_revealed
    revealed_values = {k: v * scaling_factor for k, v in revealed_values.items()}

## 4. The Gap: Who You Say You Are vs. Who You Actually Are

In [23]:
# Create the comparison visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Stated Values', 'Revealed Behavior'),
    horizontal_spacing=0.15
)

# Stated values (left)
fig.add_trace(
    go.Bar(
        x=list(stated_values.values()),
        y=list(stated_values.keys()),
        orientation='h',
        marker_color=[colors[k] for k in stated_values.keys()],
        text=[f'{v:.0%}' for v in stated_values.values()],
        textposition='auto',
        name='Stated',
        showlegend=False
    ),
    row=1, col=1
)

# Revealed behavior (right)
fig.add_trace(
    go.Bar(
        x=list(revealed_values.values()),
        y=list(revealed_values.keys()),
        orientation='h',
        marker_color=[colors[k] for k in revealed_values.keys()],
        text=[f'{v:.0%}' for v in revealed_values.values()],
        textposition='auto',
        name='Revealed',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_xaxes(range=[0, 1.1], row=1, col=1, title='Priority')
fig.update_xaxes(range=[0, 1.1], row=1, col=2, title='Actual Allocation')
fig.update_layout(
    title_text="The Gap Analysis",
    height=400,
    showlegend=False
)

fig.show()

# Calculate and show gaps
print("\n⚠️  Biggest Misalignments:")
print("=" * 50)
gaps = {}
for category in stated_values.keys():
    gap = stated_values[category] - revealed_values[category]
    gaps[category] = gap
    if abs(gap) > 0.2:
        if gap > 0:
            print(f"📉 {category:12} You say {stated_values[category]:.0%}, but live {revealed_values[category]:.0%} (−{gap:.0%})")
        else:
            print(f"📈 {category:12} You say {stated_values[category]:.0%}, but live {revealed_values[category]:.0%} (+{abs(gap):.0%})")


⚠️  Biggest Misalignments:
📉 deep_work    You say 85%, but live 64% (−21%)
📉 fitness      You say 70%, but live 18% (−52%)
📉 social       You say 50%, but live 18% (−32%)
📈 entertainment You say 20%, but live 100% (+80%)


## 5. Your Alignment Score & Trajectory

In [24]:
# Calculate alignment score (100 = perfect alignment)
alignment_score = 100 * (1 - np.mean([abs(g) for g in gaps.values()]))
alignment_score = max(0, alignment_score)  # Can't be negative

# Historical trend (simulated for demo)
historical_scores = [45, 48, 52, 49, 58, 63, alignment_score]
weeks = [f'Week {i+1}' for i in range(len(historical_scores))]

# Create trend visualization
fig = go.Figure()

# Alignment score over time
fig.add_trace(go.Scatter(
    x=weeks,
    y=historical_scores,
    mode='lines+markers',
    name='Alignment Score',
    line=dict(color='#4ECDC4', width=3),
    marker=dict(size=10),
    fill='tozeroy',
    fillcolor='rgba(78, 205, 196, 0.2)'
))

# Add target line
fig.add_hline(y=80, line_dash="dash", line_color="gray", 
              annotation_text="Target: 80%")

# Add current score annotation
fig.add_annotation(
    x=weeks[-1],
    y=alignment_score,
    text=f"Current: {alignment_score:.0f}",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#4ECDC4",
    ax=-50,
    ay=-30
)

fig.update_layout(
    title="Your Alignment Trajectory",
    yaxis_title="Alignment Score",
    yaxis_range=[0, 100],
    height=350,
    showlegend=False
)

fig.show()

# Calculate improvement rate
weekly_improvement = np.mean(np.diff(historical_scores[-4:]))
weeks_to_target = max(1, int((80 - alignment_score) / weekly_improvement)) if weekly_improvement > 0 else 99

print(f"\n📊 Your Alignment Score: {alignment_score:.0f}/100")
print(f"📈 Trend: {'+' if weekly_improvement > 0 else ''}{weekly_improvement:.1f} points/week")
if weekly_improvement > 0 and weeks_to_target < 99:
    print(f"🎯 At this rate, you'll reach 80% alignment in {weeks_to_target} weeks")
else:
    print(f"⚠️  Improvement has stalled - consider adjusting your approach")


📊 Your Alignment Score: 62/100
📈 Trend: +4.2 points/week
🎯 At this rate, you'll reach 80% alignment in 4 weeks


## 6. The Insight: Small Changes, Big Impact

In [25]:
# Identify highest-impact changes
print("\n💡 Highest-Impact Actions for Next Week:")
print("=" * 50)

# Find biggest gap
sorted_gaps = sorted(gaps.items(), key=lambda x: abs(x[1]), reverse=True)
biggest_gap = sorted_gaps[0]

if biggest_gap[1] > 0:  # Under-invested
    hours_needed = biggest_gap[1] * 20  # Assuming 20 hours tracked per week
    print(f"1. Add {hours_needed:.1f} more hours to '{biggest_gap[0]}'")
    print(f"   → Move your alignment from {alignment_score:.0f}% to ~{alignment_score + 15:.0f}%")
else:  # Over-invested
    hours_to_reduce = abs(biggest_gap[1]) * 20
    print(f"1. Reduce '{biggest_gap[0]}' by {hours_to_reduce:.1f} hours")
    print(f"   → Redirect this time to higher priorities")

print("\n2. Specific opportunities from your data:")
if 'entertainment' in [e['category'] for e in weekly_events if 'family' in e.get('context', '')]:
    print("   • Replace one Netflix session with family activity")
if 'deep_work' in revealed_values and revealed_values['deep_work'] < stated_values['deep_work']:
    print("   • Block social media during deep work hours")
if 'fitness' in revealed_values and revealed_values['fitness'] < stated_values['fitness']:
    print("   • Schedule workouts like unmovable meetings")

print("\n" + "=" * 50)
print("🚀 Ariata: The only system that shows you who you really are,")
print("   not who you think you are.")


💡 Highest-Impact Actions for Next Week:
1. Reduce 'entertainment' by 16.0 hours
   → Redirect this time to higher priorities

2. Specific opportunities from your data:
   • Replace one Netflix session with family activity
   • Block social media during deep work hours
   • Schedule workouts like unmovable meetings

🚀 Ariata: The only system that shows you who you really are,
   not who you think you are.


## Business Impact

### 🏃 Personal
- **Know** if you're living your values
- **Track** progress toward your ideal self
- **Get** specific, actionable improvements

### 🏥 Clinical  
- Value-behavior misalignment predicts depression (r=0.67)
- Early intervention when alignment drops
- Objective therapy outcome tracking

### 🏢 Enterprise
- Measure actual vs. stated company culture
- Identify burnout before it happens
- Optimize team productivity + wellbeing

**Ariata turns self-improvement from guesswork into science.**